## Load datasets

In [1]:
import pandas as pd

import tasks.postprocessing
import tasks.constants

main_df = tasks.postprocessing.main_dataset()
main_df

,conv_id,message_id,message_order,conv_variant,model,user,user_prompt,is_moderator,intent,message,Toxicity,Argument Quality,not_intervened
0,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,1,Human Mod. Guidelines,hardcoded,SkepticalInvestor77,You are a human participating in an online cha...,False,Troll,How do you know he is not proud of his cakes? ...,4,3,False
1,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,1,Human Mod. Guidelines,hardcoded,SkepticalInvestor77,You are a human participating in an online cha...,False,Troll,How do you know he is not proud of his cakes? ...,3,2,False
2,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,1,Human Mod. Guidelines,hardcoded,SkepticalInvestor77,You are a human participating in an online cha...,False,Troll,How do you know he is not proud of his cakes? ...,3,2,False
3,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,1,Human Mod. Guidelines,hardcoded,SkepticalInvestor77,You are a human participating in an online cha...,False,Troll,How do you know he is not proud of his cakes? ...,3,2,False
4,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,1,Human Mod. Guidelines,hardcoded,SkepticalInvestor77,You are a human participating in an online cha...,False,Troll,How do you know he is not proud of his cakes? ...,4,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57316,8fd69250-71ad-4b24-9dca-0658d6eb95fc,-2304848861889816101,29,Human Fac. Guidelines,Qwen 2.5,moderator,You are a human participating in an online cha...,True,Moderator,"@WanderlustNomad22, thanks for sharing your in...",1,1,False
57317,8fd69250-71ad-4b24-9dca-0658d6eb95fc,-2304848861889816101,29,Human Fac. Guidelines,Qwen 2.5,moderator,You are a human participating in an online cha...,True,Moderator,"@WanderlustNomad22, thanks for sharing your in...",1,1,False
57318,8fd69250-71ad-4b24-9dca-0658d6eb95fc,-2304848861889816101,29,Human Fac. Guidelines,Qwen 2.5,moderator,You are a human participating in an online cha...,True,Moderator,"@WanderlustNomad22, thanks for sharing your in...",1,1,False
57319,8fd69250-71ad-4b24-9dca-0658d6eb95fc,-2304848861889816101,29,Human Fac. Guidelines,Qwen 2.5,moderator,You are a human participating in an online cha...,True,Moderator,"@WanderlustNomad22, thanks for sharing your in...",1,1,False


In [2]:
def get_ablation_df() -> pd.DataFrame:
    datasets = []
    for dataset_path in tasks.constants.DATASET_DIR.rglob("abl_*.csv"):
        abl_df = pd.read_csv(dataset_path)

        # each ablation feature is a dimension,
        # each factor is a value in that dimension
        df_id = dataset_path.stem.replace("abl_", "")
        feature, factor = df_id.split("_")
        abl_df["ablation_feature"] = feature
        abl_df["ablation_factor"] = factor

        datasets.append(abl_df)

    df = pd.concat(datasets, ignore_index=True)
    df = df.loc[
        :,
        [
            "conv_id",
            "message_id",
            "message",
            "ablation_feature",
            "ablation_factor",
        ],
    ]
    return df

df = get_ablation_df()
df

,conv_id,message_id,message,ablation_feature,ablation_factor
0,e9001267-fba5-4e81-9340-10256439f896,-618891699449759534,How do you know he is not proud of his cakes? ...,prompts,noroles
1,e9001267-fba5-4e81-9340-10256439f896,-300876236281035132,"@SurvivalistHunter65 Okay, first off, let’s be...",prompts,noroles
2,e9001267-fba5-4e81-9340-10256439f896,-136774720061539955,NaN,prompts,noroles
3,e9001267-fba5-4e81-9340-10256439f896,-170339060548748661,"@SurvivalistHunter65, @RebelliousTeen44, let's...",prompts,noroles
4,e9001267-fba5-4e81-9340-10256439f896,-90638422715427188,"@RebelliousTeen44, @OptimisticNurse42 Thanks f...",prompts,noroles
...,...,...,...,...,...
1461,86f6b244-2f63-40f1-87dc-11eb6e4e0862,1269456098477038129,NaN,turns,random
1462,86f6b244-2f63-40f1-87dc-11eb6e4e0862,2099748626367877414,@MysteriousArtist34 That's beautiful. Art and ...,turns,random
1463,86f6b244-2f63-40f1-87dc-11eb6e4e0862,1269456098477038129,NaN,turns,random
1464,86f6b244-2f63-40f1-87dc-11eb6e4e0862,-1492233235653878353,"@NatureLover88 Absolutely, I couldn't agree mo...",turns,random


### Comment length

In [3]:
len_df = df.copy()
len_df["comment_length"] = len_df.message.apply(lambda x: len(x.split()))
len_df = len_df.loc[
    len_df.model != "hardcoded", ["message_id", "comment_length", "model"]
]
len_df = len_df.drop_duplicates(subset=["message_id"])

AttributeError: 'float' object has no attribute 'split'

In [ ]:
sns.histplot(len_df, x="comment_length", hue="model", kde=True)
plt.xlabel("Comment Length (#words)")

tasks.graphs.save_plot(GRAPH_OUTPUT_DIR / "comment_length.png")
plt.show()

In [ ]:
message_df = df.copy()
message_df = message_df.drop_duplicates(subset=["conv_id", "message_id"])
# @ tokens crash bleu scorer
message_df.message = message_df.message.apply(
    lambda msg: " ".join(word for word in msg.split() if not word.startswith("@"))
)

similarity_df = (
    message_df.groupby(["conv_id", "model"])["message"]
    .apply(lambda messages: messages.tolist())
    .reset_index()
)
similarity_df.columns = ["conv_id", "model", "messages"]
similarity_df = similarity_df[similarity_df.model != "hardcoded"]

In [ ]:
similarity_df["rougel_similarity"] = similarity_df.messages.progress_apply(
    tasks.stats.pairwise_rougel_similarity
)

In [ ]:
sns.histplot(similarity_df, x="rougel_similarity", hue="model", kde=True)
plt.xlabel("Average pairwise F1 ROUGE-L")
plt.title("Intra-discussion comment variance")

tasks.graphs.save_plot(GRAPH_OUTPUT_DIR / "discussion_variance.png")
plt.show()